In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import pickle

sys.path.append("..")
from dm21cm.dm_params import DMParams

sys.path.append(os.environ['DH_DIR'])
from darkhistory.main import evolve as evolve_DH
from darkhistory.config import load_data

In [2]:
tf_version = 'zf01'

load_data('dep_tf', prefix=f"{os.environ['DH_DATA_DIR']}/../DHdata_v1_1_{tf_version}")
load_data('ics_tf', prefix=f"{os.environ['DH_DATA_DIR']}/../DHdata_v1_1_{tf_version}");

****** Loading transfer functions... ******
Using data at /n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1/../DHdata_v1_1_zf01
    for propagating photons...  

 Done!
    for low-energy photons...  Done!
    for low-energy electrons...  Done!
    for high-energy deposition...  Done!
    for total upscattered CMB energy rate...  Done!
****** Loading complete! ******
****** Loading transfer functions... ******
    for inverse Compton (Thomson)...  Done!
    for inverse Compton (relativistic)...  Done!
    for inverse Compton (energy loss)...  Done!
****** Loading complete! ******


In [5]:
# phph
dm_params = DMParams(
    mode = 'decay',
    primary = 'phot_delta',
    m_DM = 1e8, # [eV]
    lifetime = 1e26, #[s]
)

In [3]:
# ee
dm_params = DMParams(
    mode = 'decay',
    primary = 'elec_delta',
    m_DM = 1e8, # [eV]
    lifetime = 1e26, #[s]
)

In [4]:
# off
dm_params = DMParams(
    mode = 'decay',
    primary = 'phot_delta',
    m_DM = 1e8, # [eV]
    lifetime = 1e50, #[s]
)

In [4]:
run_name = 'xc_ee_noHe'

soln = evolve_DH(
    DM_process=dm_params.mode, mDM=dm_params.m_DM,
    primary=dm_params.primary,
    sigmav=dm_params.sigmav, lifetime=dm_params.lifetime,
    struct_boost=dm_params.struct_boost,
    start_rs=3000, end_rs=4.+1, coarsen_factor=12, verbose=1,
    # use fake reionization to get case-A recomb. coeff.
    reion_switch=True, reion_rs=47.,
    photoion_rate_func=[lambda x: 0., lambda x: 0., lambda x: 0.],
    photoheat_rate_func=[lambda x: 0., lambda x: 0., lambda x: 0.],
    cross_check_21cmfast=True,
    debug_no_bath=False,
    debug_inject_ST_xray=False,
)

pickle.dump(soln, open(f"{os.environ['DM21CM_DIR']}/outputs/dh/{run_name}_soln.p", 'wb'))

Loading time: 0.022 s


  0%|          | 0/534 [00:00<?, ?it/s]WARNING:root:Cross checking 21cmfast!


Initialization time: 0.301 s


  0%|          | 1/534 [00:00<05:13,  1.70it/s]/n/home07/yitians/darkhistory/DarkHistory/darkhistory/spec/spectrum.py:861: RuntimeWarning: The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.
  warnings.warn("The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.", RuntimeWarning)
/n/home07/yitians/darkhistory/DarkHistory/darkhistory/history/tla.py:640: RuntimeWarning: invalid value encountered in arctanh
  _init_cond[2] = np.arctanh(2/chi * (_init_cond[2] - chi/2))
/n/home07/yitians/.conda/envs/dm21cm/lib/python3.11/site-packages/scipy/integrate/_odepack_py.py:248: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
  0%|          | 2/534 [00:01<05:51,  1.52it/s]

 lsoda--  at start of problem, too much accuracy     
       requested for precision of machine..  see tolsf (=r1) 
      in above message,  r1 =                  NaN


...

 lsoda--  at start of problem, too much accuracy     
       requested for precision of machine..  see tolsf (=r1) 
      in above message,  r1 =                  NaN
 lsoda--  repeated occurrences of illegal input      


  9%|▉         | 49/534 [00:11<01:42,  4.72it/s]

 lsoda--  at start of problem, too much accuracy     
       requested for precision of machine..  see tolsf (=r1) 
      in above message,  r1 =                  NaN
 lsoda--  repeated occurrences of illegal input      


 64%|██████▍   | 344/534 [01:16<00:41,  4.55it/s]WARNING:root:Setting coarsen_factor=1!
573it [01:42,  5.60it/s]                         


Main loop time: 102.170 s
